Just was practicing with this code

In [ ]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from obspy.signal.trigger import classic_sta_lta

# Define input and output directories
input_folder = "YOUR_INPUT_FOLDER_PATH"  # Update with your folder path
output_folder = "output_plots"

# Ensure the output directory exists
os.makedirs(output_folder, exist_ok=True)

# Function to process a single file and generate a plot
def process_jsonl_file(file_path):
    # Read data from JSONL file
    with open(file_path, "r") as file:
        for line in file:
            record = json.loads(line)
            device_time = datetime.utcfromtimestamp(record["device_t"])  # Convert to GMT
            sr = record["sr"]  # Sampling rate
            device_id = record.get("device_id", "unknown")  # Get device ID

            # Extract Date in YYYY-MM-DD format
            date_str = device_time.strftime('%Y-%m-%d')

            # Generate time values
            duration = len(record["x"]) / sr
            time_values = [(device_time - timedelta(seconds=(duration - i / sr))).strftime('%H:%M:%S.%f')[:-3]
                           for i in range(len(record["x"]))]

            # Convert to numpy arrays
            x_data = np.array(record["x"])
            y_data = np.array(record["y"])
            z_data = np.array(record["z"])
            time_data = np.array(time_values)

            # Compute STA/LTA ratios
            sta_window = int(1 * sr)  # 1-second STA window
            lta_window = int(10 * sr)  # 10-second LTA window
            sta_lta_x = classic_sta_lta(x_data, sta_window, lta_window)
            sta_lta_y = classic_sta_lta(y_data, sta_window, lta_window)
            sta_lta_z = classic_sta_lta(z_data, sta_window, lta_window)

            # Create output subfolder structure
            output_subfolder = os.path.join(output_folder, date_str, str(device_id))
            os.makedirs(output_subfolder, exist_ok=True)

            # Reduce the number of x-axis labels
            step = int(sr * 10)  # Show label every 10 seconds
            time_labels = time_data[::step]
            indices = np.arange(len(time_data))[::step]

            # Create plots
            fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

            def plot_accel_with_sta_lta(ax, data, sta_lta, color, label):
                ax.plot(time_data, data, color=color, label=f"{label}-axis")
                ax.set_ylabel(f"Acceleration ({label})")
                ax.legend(loc="upper left")

                ax2 = ax.twinx()
                ax2.plot(time_data, sta_lta, color='black', alpha=0.7, linestyle="--", label="STA/LTA Ratio")
                ax2.set_ylabel("STA/LTA Ratio")
                ax2.legend(loc="upper right")

            plot_accel_with_sta_lta(axes[0], x_data, sta_lta_x, 'orange', "X")
            plot_accel_with_sta_lta(axes[1], y_data, sta_lta_y, 'yellow', "Y")
            plot_accel_with_sta_lta(axes[2], z_data, sta_lta_z, 'green', "Z")
            axes[2].set_xlabel("Time (GMT)")
            axes[2].set_xticks(indices)
            axes[2].set_xticklabels(time_labels, rotation=45)

            plt.tight_layout()

            # Save plot in subfolder
            plot_filename = os.path.join(output_subfolder, f"{device_id}_plot.png")
            plt.savefig(plot_filename)
            plt.close()

            print(f"Saved plot: {plot_filename}")

# Process all JSONL files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jsonl"):
        process_jsonl_file(os.path.join(input_folder, filename))

print("All files processed and plots saved.")


This is the actual working code for the tree structure of the saved file

In [2]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from obspy.signal.trigger import classic_sta_lta

# Define input and output directories
input_root = "AWS EARTHQUAKE DATASET"
output_root = "output plot save"

# Traverse all subdirectories
for date_folder in os.listdir(input_root):
    date_path = os.path.join(input_root, date_folder)
    
    if not os.path.isdir(date_path):  
        continue  # Skip if not a directory
    
    for device_folder in os.listdir(date_path):
        device_path = os.path.join(date_path, device_folder)

        if not os.path.isdir(device_path):
            continue  # Skip if not a directory
        
        for json_file in os.listdir(device_path):
            if not json_file.endswith(".jsonl"):
                continue  # Process only JSONL files
            
            file_path = os.path.join(device_path, json_file)
            
            # Initialize lists for data and timestamps
            x_data, y_data, z_data, time_data = [], [], [], []

            # Read and process JSONL file
            with open(file_path, "r") as file:
                for line in file:
                    record = json.loads(line)
                    device_time = datetime.utcfromtimestamp(record["device_t"])  # Convert to GMT
                    sr = record["sr"]  # Sampling rate

                    # Generate time values
                    duration = len(record["x"]) / sr
                    time_values = [(device_time - timedelta(seconds=(duration - i / sr))).strftime('%H:%M:%S.%f')[:-3]
                                   for i in range(len(record["x"]))]

                    x_data.extend(record["x"])
                    y_data.extend(record["y"])
                    z_data.extend(record["z"])
                    time_data.extend(time_values)

            # Convert to numpy arrays
            x_data, y_data, z_data, time_data = map(np.array, [x_data, y_data, z_data, time_data])

            # Compute STA/LTA ratios
            sta_window = int(1 * sr)  # Short-term window
            lta_window = int(10 * sr)  # Long-term window

            sta_lta_x = classic_sta_lta(x_data, sta_window, lta_window)
            sta_lta_y = classic_sta_lta(y_data, sta_window, lta_window)
            sta_lta_z = classic_sta_lta(z_data, sta_window, lta_window)

            # Reduce x-axis label density
            step = int(sr * 10)  
            time_labels = time_data[::step]
            indices = np.arange(len(time_data))[::step]

            # Create subplots for x, y, and z axes
            fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

            def plot_accel_with_sta_lta(ax, data, sta_lta, color, label):
                ax.plot(time_data, data, color=color, label=f"{label}-axis")
                ax.set_ylabel(f"Acceleration ({label})")
                ax.legend(loc="upper left")

                ax2 = ax.twinx()
                ax2.plot(time_data, sta_lta, color='black', alpha=0.7, linestyle="--", label="STA/LTA Ratio")
                ax2.set_ylabel("STA/LTA Ratio")
                ax2.legend(loc="upper right")

            # Plot acceleration and STA/LTA
            plot_accel_with_sta_lta(axes[0], x_data, sta_lta_x, 'orange', "X")
            plot_accel_with_sta_lta(axes[1], y_data, sta_lta_y, 'yellow', "Y")
            plot_accel_with_sta_lta(axes[2], z_data, sta_lta_z, 'green', "Z")
            axes[2].set_xlabel("Time (GMT)")
            axes[2].set_xticks(indices)
            axes[2].set_xticklabels(time_labels, rotation=45)

            plt.tight_layout()

            # Define output path
            output_dir = os.path.join(output_root, date_folder, device_folder)
            os.makedirs(output_dir, exist_ok=True)
            output_file = os.path.join(output_dir, f"{json_file}.png")

            # Save plot
            plt.savefig(output_file)
            plt.close()

print("Processing and saving completed!")


C:\Users\Ayush_trainee\AppData\Local\Temp\ipykernel_9008\3341303706.py:38: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  device_time = datetime.utcfromtimestamp(record["device_t"])  # Convert to GMT
C:\Users\Ayush_trainee\AppData\Local\Temp\ipykernel_9008\3341303706.py:38: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  device_time = datetime.utcfromtimestamp(record["device_t"])  # Convert to GMT
C:\Users\Ayush_trainee\AppData\Local\Temp\ipykernel_9008\3341303706.py:38: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to

Processing and saving completed!
